Nakonec nepoužito

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
import time
import csv

In [6]:


def get_job_offers(job_name, driver):
    # Základní URL s formátem pro hledání pozic
    base_url = 'https://nofluffjobs.com/pl/?lang=en&criteria=keyword%3D"{job_name}"'
    driver.get(base_url.format(job_name=job_name.replace(' ', '%20')))
    
    time.sleep(1)

    try:
        cookies_btn = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
        cookies_btn.click()
        time.sleep(1)
    except Exception as e:
        print('Cookies button not found or already accepted.')

    try:
        notifications_btn = driver.find_element(By.CSS_SELECTOR, '.notification__close')
        notifications_btn.click()
        time.sleep(1)
    except Exception as e:
        print('Notification close button not found or already closed.')

    time.sleep(1)

    # Klikání na tlačítko "Načíst další" (Load more)
    while True:
        try:
            load_more_button = driver.find_element(By.CSS_SELECTOR, '.tw-btn-xl')  # Změň podle skutečného selektoru tlačítka
            load_more_button.click()
            time.sleep(3)  # Počkej, až se načtou nové nabídky
        except Exception as e:
            print('No more pages or button not found.')
            break  # Pokud tlačítko není k dispozici, ukonči cyklus

    # Sběr všech odkazů na pracovní nabídky
    all_job_links = []

    # Získání všech odkazů na pracovní nabídky
    job_links = driver.find_elements(By.CSS_SELECTOR, 'a.posting-list-item')
    for link in job_links:
        job_url = link.get_attribute('href')
        job_title = link.text.strip()  # Získání textu odkazu (titul pracovní nabídky)
        all_job_links.append((job_title, job_url))

    return all_job_links

# Inicializace WebDriveru
service = Service('./geckodriver.exe')
driver = webdriver.Firefox()
driver.implicitly_wait(10)

# Zadej pozici, kterou chceš zpracovat
job_title = 'data scientist'

# Sběr dat pro konkrétní pracovní pozici
job_data = get_job_offers(job_title, driver)

# Zavření WebDriveru
driver.quit()

# Výpis výsledků (pro testování)
for job in job_data:
    print(job)

def export_to_html(job_data, filename='job_listings.html'):
    # Vytvoření základní HTML struktury
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Job Listings</title>
    </head>
    <body>
        <h1>Job Listings</h1>
        <ul>
    """
    
    # Přidání jednotlivých odkazů do HTML struktury
    for job in job_data:
        job_title, job_url = job  # Rozbalení tuple
        html_content += f'<li><a href="{job_url}">{job_title}</a></li>\n'
    
    # Uzavření HTML struktury
    html_content += """
        </ul>
    </body>
    </html>
    """
    
    # Uložení HTML souboru
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(html_content)

    print(f"HTML file '{filename}' created successfully.")

def export_to_csv(job_data, filename='job_listings.csv'):
    # Uložení dat do CSV
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Job Title', 'Job URL'])
        writer.writerows(job_data)
    
    print(f"CSV file '{filename}' created successfully.")


# Zadej pozici, kterou chceš zpracovat
job_title = 'data scientist'


# Export dat do HTML a CSV
export_to_html(job_data, filename='job_listings_data_scientist.html')
export_to_csv(job_data, filename='job_listings_data_scientist.csv')

No more pages or button not found.
('Data Scientist\nNEW\nData\n  AI\n  Machine learning\n  Python\n25 200 – 28 560 PLN\nRemote\n +1\nConnectis_', 'https://nofluffjobs.com/pl/job/data-scientist-connectis--remote-2')
('Lead Data Scientist / ML Engineer\nNEW\nSUPPORT\nAI\n  Python\n  ML\n  Data science\n19 320 – 28 560 PLN\nRemote\n +5\nAddepto', 'https://nofluffjobs.com/pl/job/lead-data-scientist-ml-engineer-addepto-remote')
('Senior Data Scientist\nNEW\nData\n  Machine learning\n  Analytical skills\n  Python\n22 400 – 28 823 PLN\nWarszawa\nAdform Sp. z o.o.', 'https://nofluffjobs.com/pl/job/senior-data-scientist-adform-warszawa')
('Senior Data Scientist\nNEW\nData\n  Python\n  Big data\n  Data science\n16 198 – 21 598 PLN\nRemote\n +1\nDATAPAO Kft.', 'https://nofluffjobs.com/pl/job/senior-data-scientist-datapao-kft--remote')
('Senior Data Scientist\nNEW\nData\n  Python\n  Hadoop\n  Machine learning\n17 000 – 22 000 PLN\nGdynia\n +4\nLink Group', 'https://nofluffjobs.com/pl/job/senior-d

In [7]:
from bs4 import BeautifulSoup
import re

def read_html_file(file_path):
    """Načte HTML obsah ze souboru."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def parse_job_listing(html):
    """Parses HTML content and extracts job information."""
    soup = BeautifulSoup(html, 'html.parser')
    job_items = soup.find_all('li')
    
    jobs = []
    
    for item in job_items:
        link = item.find('a')
        if not link:
            continue
        
        job_url = link['href']
        job_title = link.text.strip()

        # Rozdělení textu na části
        parts = job_title.split('\n')
        if len(parts) < 5:
            continue
        
        # Extrakce jednotlivých částí
        job_name = parts[0].strip()
        # Technologie jsou obvykle mezi názvem práce a platovými informacemi
        technology = [tech.strip() for tech in parts[2:4] if tech.strip()]
        salary_str = parts[-3].strip()
        location_city = parts[-2].strip()
        company_name = parts[-1].strip()

        # Extrakce informací o platu
        salary_match = re.match(r'([\d\s]+)–([\d\s]+) (\w+)', salary_str)
        if salary_match:
            salary_low, salary_high, salary_currency = salary_match.groups()
            salary_low = salary_low.strip().replace(' ', '')
            salary_high = salary_high.strip().replace(' ', '')
        else:
            salary_low, salary_high, salary_currency = None, None, None

        job_data = {
            'name': job_name,
            'company': company_name,
            'technology': technology,
            'job': job_name,
            'location': {'city': location_city, 'country': 'Poland'},  # Assuming the country is Poland
            'salary': {'low': salary_low, 'high': salary_high, 'currency': salary_currency}
        }

        # Vytvoření řetězce se `;` jako oddělovačem
        formatted_output = "; ".join([f"{key}: {value}" for key, value in job_data.items()])
        
        jobs.append(job_data)
    
    return jobs

# Cesta k HTML souboru
file_path = 'job_listings_data_scientist.html'

# Načtení HTML souboru
html_content = read_html_file(file_path)

# Parsování HTML a získání dat
jobs = parse_job_listing(html_content)

# Výpis výsledků
for job in jobs:
    print(job)

{'name': 'Data Scientist', 'company': 'Connectis_', 'technology': ['Data', 'AI'], 'job': 'Data Scientist', 'location': {'city': '+1', 'country': 'Poland'}, 'salary': {'low': None, 'high': None, 'currency': None}}
{'name': 'Lead Data Scientist / ML Engineer', 'company': 'Addepto', 'technology': ['SUPPORT', 'AI'], 'job': 'Lead Data Scientist / ML Engineer', 'location': {'city': '+5', 'country': 'Poland'}, 'salary': {'low': None, 'high': None, 'currency': None}}
{'name': 'Senior Data Scientist', 'company': 'Adform Sp. z o.o.', 'technology': ['Data', 'Machine learning'], 'job': 'Senior Data Scientist', 'location': {'city': 'Warszawa', 'country': 'Poland'}, 'salary': {'low': '22400', 'high': '28823', 'currency': 'PLN'}}
{'name': 'Senior Data Scientist', 'company': 'DATAPAO Kft.', 'technology': ['Data', 'Python'], 'job': 'Senior Data Scientist', 'location': {'city': '+1', 'country': 'Poland'}, 'salary': {'low': None, 'high': None, 'currency': None}}
{'name': 'Senior Data Scientist', 'company

In [8]:
from bs4 import BeautifulSoup
import re
import csv

def read_html_file(file_path):
    """Načte HTML obsah ze souboru."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def parse_job_listing(html):
    """Parses HTML content and extracts job information."""
    soup = BeautifulSoup(html, 'html.parser')
    job_items = soup.find_all('li')
    
    jobs = []
    
    for item in job_items:
        link = item.find('a')
        if not link:
            continue
        
        job_url = link['href']
        job_title = link.text.strip()

        # Rozdělení textu na části
        parts = job_title.split('\n')
        if len(parts) < 5:
            continue
        
        # Extrakce jednotlivých částí
        job_name = parts[0].strip()
        technology = [tech.strip() for tech in parts[2:4] if tech.strip()]
        salary_str = parts[-3].strip()
        location_city = parts[-2].strip()
        company_name = parts[-1].strip()

        # Extrakce informací o platu
        salary_match = re.match(r'([\d\s]+)–([\d\s]+) (\w+)', salary_str)
        if salary_match:
            salary_low, salary_high, salary_currency = salary_match.groups()
            salary_low = salary_low.strip().replace(' ', '')
            salary_high = salary_high.strip().replace(' ', '')
        else:
            salary_low, salary_high, salary_currency = None, None, None

        job_data = {
            'name': job_name,
            'company': company_name,
            'technology': '; '.join(technology),  # Spojení technologií do jednoho řetězce
            'job': job_name,
            'location': {'city': location_city, 'country': 'Poland'},  # Assuming the country is Poland
            'salary': {'low': salary_low, 'high': salary_high, 'currency': salary_currency}
        }
        
        jobs.append(job_data)
    
    return jobs

def save_to_csv(jobs, output_file):
    """Uloží seznam pracovních nabídek do CSV souboru."""
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'company', 'technology', 'job', 'location_city', 'location_country', 'salary_low', 'salary_high', 'salary_currency']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')

        writer.writeheader()
        for job in jobs:
            writer.writerow({
                'name': job['name'],
                'company': job['company'],
                'technology': job['technology'],
                'job': job['job'],
                'location_city': job['location']['city'],
                'location_country': job['location']['country'],
                'salary_low': job['salary']['low'],
                'salary_high': job['salary']['high'],
                'salary_currency': job['salary']['currency']
            })

# Cesta k HTML souboru
file_path = 'job_listings_data_scientist.html'

# Načtení HTML souboru
html_content = read_html_file(file_path)

# Parsování HTML a získání dat
jobs = parse_job_listing(html_content)

# Uložení dat do CSV
output_csv_file = 'job_listings_data_scientist2.csv'
save_to_csv(jobs, output_csv_file)

print(f"Data byla uložena do {output_csv_file}.")

Data byla uložena do job_listings_data_scientist2.csv.
